In [3]:
#%%time #JypyterLab commad
import simpy
import random
import math
import pandas as pd
import time
import heapq


# Minimal print output (in python)?
max_print_out = False

# Create excel file?
create_excel = False
save_every = 60*10 # Save every x minutes  # set number_of_simulations + 1 otherwise cutoff 

# Simulation time
number_of_simulations = 10  # How many simulations do you want to run?
# Simulate from 8 AM till 6 PM
simulation_time = 3600 * 10 + 1  # Define the maximum time limit for the whole simulation in seconds (+1 for saving to excel)
order_time_limit = 3600 * 10 - 60 * 30  # Define the maximum time limit for incoming orders in seconds (Longest possible order Trip = 1820m)

# Define car data
speed = 1.6  # Car speed in m/s
fleet_size = 6 # Number of cars in the fleet
initially_available = 3 # How many are initially available?
second_group = 3600 * 2.5 # When will the second group be available?
charging_time = 3600 * 4.5 # Define recharging time in seconds
battery_capacity = 480 * 1.5  # Define battery capacity in Wh
charging_threshold = 160 # Define battery charging threshold in Wh (L1 150 Wh; L5 135 Wh; L1/3/5/10/12 110 Wh)
service_time_min = 30 # Minimum servecing time for packages and charging in seconds
service_time_max = 120 # Maximum servecing time for packages and chargings in seconds
# Predictive formula for total orders: order_time_limit / (order_time_max - order_time min / 2) + "busy hours"-orders
car_idle_time = 30 # The timeinterval in which an idle car checks for new orders
busy_hour_factor = 0.5 # Set this to < 1 to generate more order in the first simulation hour
order_time_min = 189 * 1 # Minimum time between orders
order_time_max = 567 * 1 # Maximum time between orders
take_additional_orders = True # Is the car allowed to service additional orders if they have the same destination?  
# Define a markup for time and distance to simulate random events (these will occur every time a car drives from one point to another)
additional_time_min = 0 # Possible minimal time delay in %
additional_time_max = 0.0 # Possible maximal time delay in %
additional_distance_min = 0 # Possible minimal distance markup in %
additional_distance_max = 0.0 # Possible maximal distance markup in %

# Which pathfinding algorithm should be used?
use_dijkstra = False
use_a_star = False
use_floyd_warshall = True

# Is there an order which needs to visit all the Locations? (TSP / Postszenario)
tsp_exists = True
tsp_mailstation_location = 'Location 1' # Mailstation Location # "Define the last station of the (collection) tour" correctly
tsp_charging_threshold = 245 # Define how much capacity the car should have to drive the TSP tour in Wh
morning_tsp_ammount = 1
morning_tsp_time = 0 # 8 AM
afternoon_tsp_ammount = 1
afternoon_tsp_time = 3600 * 7 # 3 PM (7 in simulation)
# Which TSP solving Algorithm?
nearest_neighbour = False
brute_force = False # Really computationally expensive! (Don't run multiple simulations. Combinde Locations in "tsp_locations" if possible. Use floyd warshall for pathfinding)
pre_defined_path = True

# Define battery consumption parameters
# Pick battery consumption calculation method 
use_fixed_consumption = True
use_formula_shao = False
use_formula_baktas = False # Same as Shao (when acceleration = 0) but originally without the efficiency parameter
use_formula_fiori = False

# Fixed rate parameters (based on technical data)
battery_consumption_rate = 0.045  # Define battery consumption rate in Wh per meter
sensor_consumption_rate = 0.0117 * (1/0.95) * 1 # Set to 0 to focus on motor consumption  # Define the sensor consumption rate in Wh per second
# Parameters for formula (Shao 2018)
efficiency_parameter = 0.85 # Efficiency parameter for everything (Electric motor and Batterie)
gravitational_constant = 9.81 # in m/s^2
rolling_resistance = 0.013 # dimensionless coefficient
angle_of_the_road = 0 # (−(π/2) ≤ α ≤ (π/2), in radians)
air_density = 1.2041 # in kg/m3
aerodynamic_drag = 1 # dimensionless coefficient 
vehicle_frontal_area = 0.5 #in m^2
mass_factor = 1.1 #dimensionless (inertia of the rotating parts (the number is from the paper))
acceleration = 0 # in m/s2
vehicle_mass = 70 # Define the weight of the car in kg 
max_cargo_mass = 30 # Define the maximum carry weight of the car in kg 

# Define locations
locations = {
    'Location A': (50, 0),
    'Location B': (50, 100),
    'Location C': (100, 120),
    'Location D': (120, 180),
    'Location E': (170, 230),
    'Location F': (240, 300),
    'Location G': (280, 340),
    'Location H': (360, 300),
    'Location I': (360, 350),
    'Location J': (360, 400),
    'Location K': (360, 450),
    'Location L': (500, 240),
    'Location M': (500, 380),
    'Location N': (480, 450),
    'Location 1': (50, 50),
    'Location 2': (180, 150),
    'Location 3': (100, 150),
    'Location 4': (190, 210),
    'Location 5': (240, 250),
    'Location 6': (330, 260),
    'Location 7': (320, 300),
    'Location 8': (320, 350),
    'Location 9': (320, 400),
    'Location 10': (320, 450),
    'Location 11': (390, 260),
    'Location 12': (410, 370),
}

# Probability of a car starting at a given location
starting_location_probabilities = {
    'Location A': 0.0,
    'Location B': 0.0,
    'Location C': 0.0,
    'Location D': 0.0,
    'Location E': 0.0,
    'Location F': 0.0,
    'Location G': 0.0,
    'Location H': 0.0,
    'Location I': 0.0,
    'Location J': 0.0,    
    'Location K': 0.0,
    'Location L': 0.0,
    'Location M': 0.0,
    'Location N': 0.0,
    'Location 5': 0.0,
    'Location 1': 10.0,
    'Location 11': 0.0,
}

# Probability of a certain location to send out an order
sending_order_probabilities = {
    'Location A': 0.1,
    'Location B': 0.1,
    'Location C': 0.1,
    'Location D': 0.1,
    'Location E': 0.1,
    'Location F': 0.1,
    'Location G': 0.1,
    'Location H': 0.1,
    'Location I': 0.1,
    'Location J': 0.1,
    'Location K': 0.1,
    'Location L': 0.1,
    'Location M': 0.1,
    'Location N': 0.1,
}

# Probability of a certain location to send out an order
receiving_order_probabilities = {
    'Location A': 0.1,
    'Location B': 0.1,
    'Location C': 0.1,
    'Location D': 0.1,
    'Location E': 0.1,
    'Location F': 0.1,
    'Location G': 0.1,
    'Location H': 0.1,
    'Location I': 0.1,
    'Location J': 0.1,
    'Location K': 0.1,
    'Location L': 0.1,
    'Location M': 0.1,
    'Location N': 0.1,
}

# TSP Locations (mark out Locations that belong together to save computational time for the brute force method)
tsp_locations = {
    'Location A',
    'Location B',
    'Location C',
    'Location D',
    'Location E',
    'Location F',
    'Location G',
    'Location H',
    'Location I',
    'Location J',
    'Location K',
    'Location L',
    'Location M',
    'Location N',
}

# Define charging stations and initialize the charging_stations dictionary
# (for i in range(ammount of charging stations)
charging_stations = {
    'Location 1': [{'id': i + 1, 'available': True, 'location': 'Location 1', 'cars': [], 'queue_length': 0, 'started_charging_timestamp': [0], 'reserved_charger_timestamp': [], 'arrived_at_charger_timestamp': [], 'finished_charging_timestamp': []} 
                   for i in range(1)],
    'Location 10': [{'id': i + 1, 'available': True, 'location': 'Location 10', 'cars': [], 'queue_length': 0, 'started_charging_timestamp': [0], 'reserved_charger_timestamp': [], 'arrived_at_charger_timestamp': [], 'finished_charging_timestamp': []} 
                   for i in range(0)],
    'Location 11': [{'id': i + 1, 'available': True, 'location': 'Location 11', 'cars': [], 'queue_length': 0, 'started_charging_timestamp': [0], 'reserved_charger_timestamp': [], 'arrived_at_charger_timestamp': [], 'finished_charging_timestamp': []} 
                   for i in range(0)],
    'Location 5': [{'id': i + 1, 'available': True, 'location': 'Location 5', 'cars': [], 'queue_length': 0, 'started_charging_timestamp': [0], 'reserved_charger_timestamp': [], 'arrived_at_charger_timestamp': [], 'finished_charging_timestamp': []} 
                   for i in range(0)],
}
 
# Define the allowed connections and real weights (distance)
allowed_connections = {
    ('Location A', 'Location 1'): 50, ('Location 1', 'Location A'): 50,
    ('Location B', 'Location 1'): 50, ('Location 1', 'Location B'): 50,
    ('Location C', 'Location 3'): 50, ('Location 3', 'Location C'): 50,
    ('Location D', 'Location 3'): 50, ('Location 3', 'Location D'): 50,
    ('Location E', 'Location 4'): 50, ('Location 4', 'Location E'): 50,
    ('Location F', 'Location 5'): 50, ('Location 5', 'Location F'): 50,
    ('Location G', 'Location 8'): 50, ('Location 8', 'Location G'): 50,
    ('Location H', 'Location 7'): 50, ('Location 7', 'Location H'): 50,
    ('Location I', 'Location 8'): 50, ('Location 8', 'Location I'): 50,
    ('Location J', 'Location 9'): 50, ('Location 9', 'Location J'): 50,
    ('Location K', 'Location 10'): 50, ('Location 10', 'Location K'): 50,
    ('Location L', 'Location 11'): 150, ('Location 11', 'Location L'): 150,
    ('Location M', 'Location 12'): 100, ('Location 12', 'Location M'): 100,
    ('Location N', 'Location 12'): 150, ('Location 12', 'Location N'): 150,
    ('Location 1', 'Location 2'): 250, ('Location 2', 'Location 1'): 250,
    ('Location 2', 'Location 3'): 80, ('Location 3', 'Location 2'): 80,
    ('Location 2', 'Location 4'): 70, ('Location 4', 'Location 2'): 70,
    ('Location 4', 'Location 5'): 100, ('Location 5', 'Location 4'): 100,
    ('Location 5', 'Location 6'): 100, ('Location 6', 'Location 5'): 100,
    ('Location 6', 'Location 7'): 50, ('Location 7', 'Location 6'): 50,
    ('Location 6', 'Location 11'): 70, ('Location 11', 'Location 6'): 70,
    ('Location 7', 'Location 8'): 50, ('Location 8', 'Location 7'): 50,
    ('Location 8', 'Location 9'): 50, ('Location 9', 'Location 8'): 50,
    ('Location 9', 'Location 10'): 50, ('Location 10', 'Location 9'): 50,
    ('Location 11', 'Location 12'): 150, ('Location 12', 'Location 11'): 150,
}


def euclidean_dist(loc1, loc2):
    x1, y1 = loc1
    x2, y2 = loc2
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

# Define the heuristic function for the A* algorithm
def heuristic(loc1, loc2):
    return euclidean_dist(locations[loc1], locations[loc2])

if use_dijkstra == True or use_a_star == True:
    # Define A* search algorithm (without the heuristic part A* is Dijkstra)
    def a_star(start_loc, end_loc):
        frontier = []
        heapq.heappush(frontier, (0, start_loc))
        came_from = {}
        cost_so_far = {}
        came_from[start_loc] = None
        cost_so_far[start_loc] = 0

        while frontier:
            current_loc = heapq.heappop(frontier)[1]

            if current_loc == end_loc:
                break

            for next_loc in locations:
                if (current_loc, next_loc) in allowed_connections:
                    new_cost = cost_so_far[current_loc] + allowed_connections[(current_loc, next_loc)]
                    if next_loc not in cost_so_far or new_cost < cost_so_far[next_loc]:
                        cost_so_far[next_loc] = new_cost
                        if use_a_star == True:
                            priority = new_cost + heuristic(next_loc, end_loc)
                        if use_dijkstra == True:
                            priority = new_cost 
                        heapq.heappush(frontier, (priority, next_loc))
                        came_from[next_loc] = current_loc

        if end_loc not in came_from:
            return None, None

        path = []
        loc = end_loc
        while loc != start_loc:
            path.append(loc)
            loc = came_from[loc]
        path.append(start_loc)
        path.reverse()

        distance = cost_so_far[end_loc]

        return path, distance

if use_floyd_warshall == True:
    class FloydWarshall:
        def __init__(self, locations, allowed_connections):
            self.locations = locations
            self.allowed_connections = allowed_connections
            self.num_locations = len(locations)
            self.distance_matrix = self.initialize_distance_matrix()
            self.next_hop_matrix = self.initialize_next_hop_matrix()

        def initialize_distance_matrix(self):
            # Initialize the distance matrix with infinity for all pairs of locations
            distance_matrix = {}
            for from_loc in self.locations:
                distance_matrix[from_loc] = {}
                for to_loc in self.locations:
                    if from_loc == to_loc:
                        distance_matrix[from_loc][to_loc] = 0
                    else:
                        distance_matrix[from_loc][to_loc] = float('inf')

            # Fill in real weights for allowed connections
            for (from_loc, to_loc), weight in self.allowed_connections.items():
                distance_matrix[from_loc][to_loc] = weight

            return distance_matrix

        def initialize_next_hop_matrix(self):
            # Initialize the next-hop matrix for path reconstruction
            next_hop_matrix = {}
            for from_loc in self.locations:
                next_hop_matrix[from_loc] = {}
                for to_loc in self.locations:
                    if from_loc != to_loc and self.distance_matrix[from_loc][to_loc] < float('inf'):
                        next_hop_matrix[from_loc][to_loc] = to_loc
                    else:
                        next_hop_matrix[from_loc][to_loc] = None

            return next_hop_matrix

        def run_floyd_warshall(self):
            if max_print_out:
                print(f' fw.run_floyd_warshal is being executed now')
            for k in self.locations:
                for i in self.locations:
                    for j in self.locations:
                        if self.distance_matrix[i][j] > self.distance_matrix[i][k] + self.distance_matrix[k][j]:
                            self.distance_matrix[i][j] = self.distance_matrix[i][k] + self.distance_matrix[k][j]
                            self.next_hop_matrix[i][j] = self.next_hop_matrix[i][k]

        def get_shortest_path(self, from_loc, to_loc):
            if from_loc not in self.locations or to_loc not in self.locations:
                return None  # Invalid locations

            if self.distance_matrix[from_loc][to_loc] == float('inf'):
                return None  # There is no path between the locations

            path = [from_loc]
            while from_loc != to_loc:
                from_loc = self.next_hop_matrix[from_loc][to_loc]
                path.append(from_loc)
            return path

        def get_shortest_distance(self, from_loc, to_loc):
            return self.distance_matrix[from_loc][to_loc]
        
    # Create an instance of the FloydWarshall class
    fw = FloydWarshall(locations, allowed_connections)  
    
    
# Define and initialize variables to track the number of deliveries, total orders, charged ammount, etc.
all_simulations_total_orders = 0
num_unfinished_orders = 0
num_deliveries_finished = 0
global_deliveries_finished = 0
total_orders = 0
num_deliveries_started = 0
total_waiting_time_till_charge = 0
all_simulations_total_waiting_time_till_charge = 0
total_charged_ammount = 0
all_simulation_total_charged_ammount = 0
total_cars_started_charging = 0
all_simulations_total_cars_started_charging = 0
total_cars_finished_charging = 0
all_simulations_total_cars_finished_charging = 0
negative_battery = 0
all_simulations_negative_battery = 0
tsp_out_per_day = 0
tsp_in_per_day = 0
tsp_out_finished = 0
tsp_in_finished = 0


# Lists for tsp 
tspcar_id = []
tspcar_origin = []
distance_to_first_point = []
time_to_first_point=[]
consumption_to_first_point = []
distance_during_tsp = []
time_of_tsp = []
consumption_during_tsp = []
tspcar_energy_consumed = []
route_taken = []
# lists for charging excel
charge_car_id = []
charge_car_origin = []
carging_station_location = []
charging_station_id = []
consumption_to_station = []
distance_to_station = []
travel_time_to_station = []
arrived_with_capacity = []
waiting_time_at_station = []


# Generate orders process
def generate_order(env, order_queue):
    global total_orders  # Declare globals
    global max_print_out
    #global sending_order_probabilities
    #global receiving_order_probabilities
    i = 0
    yield env.timeout(random.uniform(order_time_min, order_time_max))  # Initial delay before the first order
    start_time = env.now
    while True:
        if env.now - start_time >= order_time_limit:
            break

        i += 1
        # Choose a pickup location based on probabilities
        pickup_location = random.choices(
            list(sending_order_probabilities.keys()),
            weights=sending_order_probabilities.values()
        )[0]

        # Choose a delivery location based on probabilities
        delivery_location = random.choices(
            list(receiving_order_probabilities.keys()),
            weights=receiving_order_probabilities.values()
        )[0]

        weight = random.uniform(0.01, 30)  # Generate a random weight between 0 and 30 [kg]

        # Ensure that pickup and delivery locations are different
        while pickup_location == delivery_location:
            delivery_location = random.choices(
                list(receiving_order_probabilities.keys()),
                weights=receiving_order_probabilities.values()
            )[0]


        order_time = env.now  # Current simulation time
        if max_print_out == True:
            print(f"Order {i}: Time {order_time}, Weight {weight} kg, Pickup {pickup_location} - Delivery {delivery_location}")

        order = {
            'id': i,
            'pickup_location': pickup_location,
            'delivery_location': delivery_location,
            'order_time': order_time,
            'weight': weight
        }

        order_queue.put(order)  # Add the order to the queue
        total_orders += 1  # Increment the total number of orders

        # Define the "busy hours" where more orders than usual will be ordered
        if 3600 * 0 <= env.now < 3600 * 1:
            busy_timeout = random.uniform(order_time_min, order_time_max) * busy_hour_factor
            yield env.timeout(busy_timeout)  # Make delay between orders shorter during "busy hours"
        else:
             yield env.timeout(random.uniform(order_time_min, order_time_max)) # Random delay between orders
        

class CarProcess:
    def __init__(self, env, car, order_queue, charging_stations):
        self.env = env
        self.car = car
        self.order_queue = order_queue 
        self.charging_stations = charging_stations 
        self.process = env.process(self.drive())
        self.charging_station_location = None  
        self.charging_station_id = None  
        self.first_of_the_queue = None
        self.charger_reservation_time = None
        self.arrived_at_charger_time = None
        self.charging_start_time = None
        self.charging_finish_time = None
        self.charge_car_origin = None
        self.consumption_to_station = None
        self.distance_to_station = None
        self.travel_time_to_Station = None


    def drive(self):
        # Declare globals
        global num_deliveries_finished
        global total_orders
        global num_deliveries_started
        global max_print_out
        global order_nr
        global additional_order_nr
        global order_time
        global additional_order_time
        global order_weight
        global additional_order_weight
        global total_order_weight
        global order_pickup_time
        global order_delivery_time
        global car_id
        global car_origin
        global order_origin
        global order_destination
        global order_energy_consumed
        global car_energy_consumed
        global order_distance_traveled
        global car_distance_traveled
        global car_pickupt_distance
        global waiting_for_order
        while True:
            # Reset Parameters to avaid repetitions
            
            #If the car is part of the second group send it into timeout to save computational resources 
            if self.car['id'] > initially_available and env.now == 0:
                yield self.env.timeout(second_group)
                
            # Check battery capacity before taking an order
            if self.car['battery_capacity'] < charging_threshold:
                if max_print_out == True:
                    print(f"Car {self.car['id']} needs to charge. Current {self.car['current_location']}. Current battery capacity: {self.car['battery_capacity']}.")
                self.car['available'] = False
                yield self.env.process(self.search_nearest_station())

            # Check availability and battery capacity
            if self.car['available'] and self.car['battery_capacity'] >= charging_threshold and total_orders > num_deliveries_started:
                order = yield self.order_queue.get()  # Get the next order from the queue
                num_deliveries_started += 1
                self.car['available'] = False  # Set the car as unavailable
                order_processing_start_time = self.env.now  # Record the time when the car accepts the order
                if max_print_out == True:
                    print(f"Car {self.car['id']} at {self.car['current_location']} accepts order {order['id']} with a remaining battery capacity of: {self.car['battery_capacity']} Wh. Time now: {env.now}")
                
                # Drive to the pickup location and track battery consumption
                pickup_location = order['pickup_location']
                delivery_location = order['delivery_location']
                car_origin_location = self.car['current_location'] # For Excel
                if use_a_star == True or use_dijkstra == True:
                    path, distance = a_star(self.car['current_location'], pickup_location)
                if use_floyd_warshall == True:
                    path = fw.get_shortest_path(self.car['current_location'], pickup_location)
                    distance = fw.get_shortest_distance(self.car['current_location'], pickup_location)
                distance_to_pickup = distance * (1 + (random.uniform(additional_distance_min, additional_distance_max)))
                yield self.env.timeout((distance_to_pickup / speed) * (1+ (random.uniform(additional_time_min, additional_time_max))))
                self.car['current_location'] = pickup_location
                if max_print_out == True:
                    print(f"Car {self.car['id']} arrived at pickup {pickup_location} after travelling {distance_to_pickup}m. Time now: {env.now}")

                # Add weight from the initial order to car
                self.car['current_package_weight'] = order['weight']
                
                # Check if another orders with the same destination exists and if they are within the weight limit
                additional_orders_list = []  # Initialize a list to store additional orders
                additional_order_ammount = 0 # Count additional orders
                if take_additional_orders == True:
                    for additional_order in list(self.order_queue.items):
                        if (
                            additional_order['pickup_location'] == pickup_location and
                            additional_order['delivery_location'] == delivery_location and
                            self.car['current_package_weight'] + additional_order['weight'] <= max_cargo_mass
                        ):
                            # This order meets the criteria, add it to the list of additional orders
                            additional_orders_list.append(additional_order)
                            self.car['current_package_weight'] += additional_order['weight']
                            # Remove the order from the queue
                            self.order_queue.items.remove(additional_order)
                            additional_order_ammount += 1

                    # Concatenate the IDs of additional orders into a string
                    additional_order_ids = ', '.join([str(order['id']) for order in additional_orders_list])
                    additional_times = ', '.join([str(order['order_time']) for order in additional_orders_list])
                    additional_weights = ', '.join([str(order['weight']) for order in additional_orders_list])
                
                # Simulate the loading process
                yield self.env.timeout(random.uniform(service_time_min, service_time_max))
                if max_print_out == True:
                    if len(additional_orders_list) == 0:
                        print(f"Car {self.car['id']} picked up initial order {order['id']} and no additional order could be processed. Cars cargo weight: {self.car['current_package_weight']} kg. Time now: {env.now}")                
                    else:
                        print(f"Car {self.car['id']} picked up initial order {order['id']} and additional order {additional_order_ids}. Cars cargo weight: {self.car['current_package_weight']} kg. Time now: {env.now}")                
                pickup_timestamp = env.now
                        
                # Drive to the pdelivery location and track battery consumption
                if use_a_star == True or use_dijkstra == True:
                    path, distance = a_star(self.car['current_location'], delivery_location)
                if use_floyd_warshall == True:
                    path = fw.get_shortest_path(self.car['current_location'], delivery_location)
                    distance = fw.get_shortest_distance(self.car['current_location'], delivery_location)
                distance_to_delivery = distance * (1+ (random.uniform(additional_distance_min, additional_distance_max)))
                yield self.env.timeout((distance_to_delivery / speed) * (1 + (random.uniform(additional_time_min, additional_time_max))))# Drive to the delivery location
                self.car['current_location'] = delivery_location
                if max_print_out == True:
                    print(f"Car {self.car['id']} arrived at delivery {delivery_location} after travelling {distance_to_delivery}m. Time now: {env.now}")

                # Simulate the unloading process
                yield self.env.timeout(random.uniform(service_time_min, service_time_max))
            
                self.car['available'] = True  # Set the car as available again
                # Increment the number of deliveries
                num_deliveries_finished += 1 + additional_order_ammount
                #print(f'additional_orders_list: {additional_order_ids} and len additional_orders_list: {len(additional_order_ids)}')

                # Calculate the time difference between accepting and finishing the order
                order_processing_finish_time = self.env.now
                time_diff = order_processing_finish_time - order_processing_start_time

                # Determine battery consumption from the last trip 
                # Calculate battery consumption using fixed values
                if use_fixed_consumption == True: 
                    car_consumption = time_diff * self.car['sensor_consumption_rate'] + (distance_to_pickup + distance_to_delivery) * self.car['battery_consumption_rate']
                    if create_excel == True:
                        to_delivery_consumption = distance_to_delivery * self.car['battery_consumption_rate']
                        to_pickup_consumption = distance_to_pickup * self.car['battery_consumption_rate']
                # Calculate battery consumption using formula
                if use_formula_shao == True:
                    total_mass = vehicle_mass + self.car['current_package_weight']
                    to_pickup_consumption = ((distance_to_pickup / speed) / 3600) * ((speed / efficiency_parameter) * (vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 2) + vehicle_mass * mass_factor * acceleration))
                    to_delivery_consumption = ((distance_to_delivery / speed) / 3600) * ((speed / efficiency_parameter) * (total_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) + total_mass * gravitational_constant * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 2) + total_mass * mass_factor * acceleration))
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    car_consumption = to_pickup_consumption + to_delivery_consumption + sensor_consumption
                if use_formula_baktas == True:
                    total_mass = vehicle_mass + self.car['current_package_weight']
                    to_pickup_consumption = ((distance_to_pickup / speed) / 3600) * (1 / efficiency_parameter) * ((vehicle_mass * acceleration * speed + vehicle_mass * gravitational_constant * speed * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 3) + vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) * speed))
                    to_delivery_consumption = ((distance_to_delivery / speed) / 3600) * (1 / efficiency_parameter) * ((total_mass * acceleration * speed + total_mass * gravitational_constant * speed * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 3) + total_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) * speed))
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    car_consumption = to_pickup_consumption + to_delivery_consumption + sensor_consumption
                if use_formula_fiori == True:
                    total_mass = vehicle_mass + self.car['current_package_weight']
                    to_pickup_consumption = ((distance_to_pickup / speed) / 3600) * (1 / efficiency_parameter) * ((vehicle_mass * acceleration + vehicle_mass * gravitational_constant * math.cos(angle_of_the_road) * ((1.75 / 1000) * (0.0328 * speed + 4.575)) + 0.5 * air_density * vehicle_frontal_area * aerodynamic_drag * speed ** 2 + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road)) * speed)
                    to_delivery_consumption = ((distance_to_delivery / speed) / 3600) * (1 / efficiency_parameter) * ((total_mass * acceleration + total_mass * gravitational_constant * math.cos(angle_of_the_road) * ((1.75 / 1000) * (0.0328 * speed + 4.575)) + 0.5 * air_density * vehicle_frontal_area * aerodynamic_drag * speed ** 2 + total_mass * gravitational_constant * math.sin(angle_of_the_road)) * speed)
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    car_consumption = to_pickup_consumption + to_delivery_consumption + sensor_consumption
                self.car['battery_capacity'] -= car_consumption
                
                # Append data in lists for the 'order' excel datasheet
                if create_excel == True:  
                    # Reset List to prevent repetitions
                    order_nr = []
                    additional_order_nr = []
                    order_time = []
                    additional_order_time = []
                    order_weight = []
                    additional_order_weight = []
                    total_order_weight = []
                    order_pickup_time = []
                    order_delivery_time = []
                    car_id = []
                    car_origin = []
                    order_origin = []
                    order_destination = []
                    order_energy_consumed = [] # arrived_at_pickup_timestamp
                    car_energy_consumed = []
                    order_distance_traveled = []
                    car_distance_traveled = []
                    car_pickupt_distance = []
                    waiting_for_order = []
                    
                    # Create a dictionary to store the data for the 'orders' sheet
                    data_sheet_orders = {
                        'Order ID': order_nr,
                        'Additional Order ID': additional_order_nr,
                        'Order Time (s)': order_time,
                        'Additional Order Time (s)': additional_order_time,
                        'Order Weight (kg)': order_weight,
                        'Additional Order Weight (kg)': additional_order_weight,
                        'Total Order Weight (kg)': total_order_weight,
                        'Order Pickup Time (s)': order_pickup_time,
                        'Order Delivery Time (s)': order_delivery_time,
                        'Car ID' : car_id,
                        'Car Origin' : car_origin, 
                        'Order Origin' : order_origin,
                        'Order Destination' : order_destination,
                        'Order Energy Consumed (Wh)': order_energy_consumed,
                        'Car Energy Consumed (Wh)': car_energy_consumed,
                        'Order Distance Traveled (m)': order_distance_traveled,
                        'Car Distance Traveled (m)': car_distance_traveled,
                        'Car Distance to Pickup (m)': car_pickupt_distance,
                        'Waiting Time for Order (s)': waiting_for_order
                    }    
                        
                    # Append data in lists for the 'order' excel datasheet
                    order_nr.append(order['id'])
                    if len(additional_orders_list) == 0:
                        additional_order_nr.append("N/A")
                    else:
                        additional_order_nr.append(additional_order_ids)
                    order_time.append(order['order_time'])
                    if len(additional_orders_list) == 0:
                        additional_order_time.append("N/A")
                    else:
                        additional_order_time.append(additional_times)
                    order_weight.append(order['weight'])
                    if len(additional_orders_list) == 0:
                        additional_order_weight.append("N/A")
                    else:
                        additional_order_weight.append(additional_weights)
                    if len(additional_orders_list) == 0:
                        total_order_weight.append(order['weight'])
                    else:
                        total_order_weight.append(self.car['current_package_weight'])
                    order_pickup_time.append(pickup_timestamp)
                    order_delivery_time.append(env.now)                    
                    car_id.append(self.car['id'])                        
                    car_origin.append(car_origin_location)                   
                    order_origin.append(pickup_location)                            
                    order_destination.append(delivery_location)
                    to_delivery_consumption_full = to_delivery_consumption + (env.now - pickup_timestamp) * self.car['sensor_consumption_rate']
                    order_energy_consumed.append(to_delivery_consumption_full)
                    car_energy_consumed.append(car_consumption)
                    order_distance_traveled.append(distance_to_delivery)
                    car_distance_traveled.append(distance_to_pickup + distance_to_delivery)
                    car_pickupt_distance.append(distance_to_pickup)
                    waiting_for_order.append(env.now - order['order_time'])
                    
                    if len(order_nr) > 0:
                        if max_print_out == True:
                            print(f'saving order data to excel. current order id :{order_nr}')
                        # Append the data to the list of data to save for the 'orders' sheet
                        data_to_save_orders.append(data_sheet_orders)
                        
                        # Save the data to the Excel file
                        save_data_to_excel(data_to_save_data, data_to_save_orders, data_to_save_tsp_out, data_to_save_tsp_in, data_to_save_charge, filename)
                
                # Print according to whether there are additional orders
                if max_print_out == True:
                    if len(additional_orders_list) == 0:
                        print(f"Car {self.car['id']} consumed {car_consumption} Wh of battery capacity during the order {order['id']}. Current battery capacity: {self.car['battery_capacity']} Wh. The car travelled {distance_to_pickup}m to the pickup {pickup_location} and {distance_to_delivery}m to the delivery {delivery_location}. The car travelled for {time_diff}s. Time now: {env.now}")
                    else:
                        print(f"Car {self.car['id']} consumed {car_consumption} Wh of battery capacity during the order {order['id']} and additional order {additional_order_ids}. Current battery capacity: {self.car['battery_capacity']} Wh. The car travelled {distance_to_pickup}m to the pickup {pickup_location} and {distance_to_delivery}m to the delivery {delivery_location}. The car travelled for {time_diff}s. Time now: {env.now}")
 
                # Reset cargo weight after all calculations are done 
                self.car['current_package_weight'] = 0 # Empty cargo weight
                
            yield self.env.timeout(car_idle_time)  # If car is idle due to lack of orders check for new orders after timeout
            if tsp_exists == True:
                yield self.env.process(self.solve_tsp())
            else:
                yield self.env.process(self.drive())
                
    
    def solve_tsp(self):
        global tsp_out_per_day
        global tsp_in_per_day
        global tsp_out_finished
        global tsp_in_finished
        # globals for Excel
        global tspcar_id
        global tspcar_origin
        global distance_to_first_point
        global time_to_first_poi
        global consumption_to_first_point
        global distance_during_tsp
        global time_of_tsp
        global consumption_during_tsp
        global tspcar_energy_consumed
        global route_taken
        # Define TSP time and ammount from Mailstation to Locations
        if not tsp_out_per_day >= morning_tsp_ammount and morning_tsp_time <= self.env.now and self.car['battery_capacity'] > tsp_charging_threshold:
            tsp_out_per_day += 1
            locations_to_visit = list(tsp_locations)
            
            # Drive to the initial point
            if use_a_star == True or use_dijkstra == True:
                path, distance = a_star(self.car['current_location'], tsp_mailstation_location)
            if use_floyd_warshall == True:
                path = fw.get_shortest_path(self.car['current_location'], tsp_mailstation_location)
                distance = fw.get_shortest_distance(self.car['current_location'], tsp_mailstation_location)
            drive_to_location = self.env.now
            distance_to_sart_tsp = distance * (1 + (random.uniform(additional_distance_min, additional_distance_max)))                                                
            yield self.env.timeout((distance_to_sart_tsp / speed) * (1+ (random.uniform(additional_time_min, additional_time_max))))
            car_origin_float = self.car['current_location']
            self.car['current_location'] = tsp_mailstation_location
            arrive_at_location = self.env.now
            time_diff = arrive_at_location - drive_to_location
            time_diff_origin = time_diff
            if use_fixed_consumption == True:
                consumption = time_diff * self.car['sensor_consumption_rate'] + (distance_to_sart_tsp) * self.car['battery_consumption_rate']
            if use_formula_shao == True:
                driving_consumption =((distance_to_sart_tsp / speed) / 3600) * ((speed / efficiency_parameter) * (vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 2) + vehicle_mass * mass_factor * acceleration))
                sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                consumption = driving_consumption + sensor_consumption
            if use_formula_baktas == True:
                    driving_consumption = ((distance_to_sart_tsp / speed) / 3600) * (1 / efficiency_parameter) *  ((vehicle_mass * acceleration * speed + vehicle_mass * gravitational_constant * speed * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 3) + vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) * speed))
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    consumption = driving_consumption + sensor_consumption
            if use_formula_fiori == True:
                    driving_consumption = ((distance_to_sart_tsp / speed) / 3600) * (1 / efficiency_parameter) * ((vehicle_mass * acceleration + vehicle_mass * gravitational_constant * math.cos(angle_of_the_road) * ((1.75 / 1000) * (0.0328 * speed + 4.575)) + 0.5 * air_density * vehicle_frontal_area * aerodynamic_drag * speed ** 2 + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road)) * speed)
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    consumption = driving_consumption + sensor_consumption
            self.car['battery_capacity'] -= consumption
            consumption_to_first_point_float = consumption
            
            # Solve TSP with nearest neighbour
            if nearest_neighbour == True:
                tour = []
                unvisited_locations = locations_to_visit.copy()
                current_location = self.car['current_location']

                while len(unvisited_locations) > 0:
                    min_distance = float('inf')
                    nearest_location = None

                    nn_distance = 0
                    for location in unvisited_locations:
                        if use_a_star == True or use_dijkstra == True:
                            path, distance = a_star(current_location, location)
                            nn_distance = distance
                        if use_floyd_warshall == True:
                            distance = fw.get_shortest_distance(current_location, location)
                            nn_distance = distance

                        if nn_distance < min_distance:
                            min_distance = nn_distance
                            nearest_location = location

                    if nearest_location:
                        tour.append(nearest_location)
                        unvisited_locations.remove(nearest_location)
                        current_location = nearest_location
                    elif not unvisited_locations:
                        if tsp_mailstation_location in tour:
                            tour.remove(tsp_mailstation_location) # Remove the Location of the mailstation because the car is already there
                        break  # Exit the loop when all locations are visited
                tour_string = ', '.join(tour)  # Convert the tour to a comma-separated string
            
            # Solve TSP with brute force
            if brute_force:
                best_tour = None
                best_distance = float('inf')
                tour = []
                best_tour = []
                unvisited_locations = locations_to_visit.copy()
                current_location = self.car['current_location']

                def brute_force_tsp(tour, unvisited_locations):
                    nonlocal best_tour, best_distance
                    if not unvisited_locations:
                        # Calculate the total distance for the current tour
                        bf_distance = 0
                        for i in range(len(tour) - 1):
                            location1 = tour[i]
                            location2 = tour[i + 1]
                            if use_floyd_warshall == True:
                                distance = fw.get_shortest_distance(location1, location2)
                                bf_distance += distance
                            if use_a_star == True or use_dijkstra == True:
                                path, distance = a_star(location1, location2)
                                bf_distance += distance

                        # Check if it's a better tour
                        if bf_distance < best_distance:
                            best_distance = bf_distance
                            best_tour = tour

                    for location in unvisited_locations:
                        new_tour = tour + [location]
                        new_unvisited_locations = unvisited_locations.copy()
                        new_unvisited_locations.remove(location)
                        
                        # Calculate the current tour distance for this branch
                        branch_distance = 0
                        for i in range(len(new_tour) - 1):
                            location1 = new_tour[i]
                            location2 = new_tour[i + 1]
                            if use_floyd_warshall == True:
                                distance = fw.get_shortest_distance(location1, location2)
                                branch_distance += distance
                            if use_a_star == True or use_dijkstra == True:
                                path, distance = a_star(location1, location2)
                                branch_distance += distance
                        
                        # Only continue the branch if the current branch distance is less than best_distance to save computational resources
                        if branch_distance < best_distance:
                            brute_force_tsp(new_tour, new_unvisited_locations)

                brute_force_tsp([current_location], locations_to_visit)
                if tsp_mailstation_location in best_tour:
                    best_tour.remove(tsp_mailstation_location) # Remove the Location of the mailstation because the car is already there
                tour_string = ', '.join(best_tour)
            
            # Define path manually
            if pre_defined_path:
                tour_string = ', '.join(['Location A', 'Location B', 'Location C', 'Location D', 'Location E', 'Location F', 'Location H', 'Location G', 'Location I', 'Location J', 'Location K', 'Location L', 'Location M', 'Location N'])

            if max_print_out == True:
                print(f"Car {self.car['id']} will do the morning tour (TSP): {tour_string} starting at the mailstation.")  # Print the final tour as a string
            # Split tour_string into individual location names
            tour = tour_string.split(', ')
        
            # Drive the tour
            tour_distance = 0
            battery_before_tour = self.car['battery_capacity']
            time_before_tour = self.env.now
            consumption_during_tsp_float = 0
            for next_location in tour:
                if use_a_star == True or use_dijkstra == True:
                    path, distance = a_star(self.car['current_location'], next_location)
                if use_floyd_warshall == True:
                    path = fw.get_shortest_path(self.car['current_location'], next_location)
                    distance = fw.get_shortest_distance(self.car['current_location'], next_location)
                distance_to_location = distance * (1+ (random.uniform(additional_distance_min, additional_distance_max)))      

                drive_to_location = self.env.now
                if max_print_out == True:
                    print(f"Car {self.car['id']} drives {distance_to_location}m from {self.car['current_location']} to {next_location} as part of its TSP tour. Time: {self.env.now}")
                yield self.env.timeout((distance_to_location / speed) * (1+ (random.uniform(additional_time_min, additional_time_max))))
                tour_distance += distance_to_location
                self.car['current_location'] = next_location
                arrive_at_location = self.env.now
                time_diff = arrive_at_location - drive_to_location

                if use_fixed_consumption == True:
                    consumption = time_diff * self.car['sensor_consumption_rate'] + distance_to_location * self.car['battery_consumption_rate']
                if use_formula_shao == True:
                    driving_consumption = ((distance_to_location / speed) / 3600) * ((speed / efficiency_parameter) * (vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 2) + vehicle_mass * mass_factor * acceleration))
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    consumption = driving_consumption + sensor_consumption
                if use_formula_baktas == True:
                    driving_consumption = ((distance_to_location / speed) / 3600) * (1 / efficiency_parameter) * ((vehicle_mass * acceleration * speed + vehicle_mass * gravitational_constant * speed * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 3) + vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) * speed))
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    consumption = driving_consumption + sensor_consumption
                if use_formula_fiori == True:
                    driving_consumption = ((distance_to_location / speed) / 3600) * (1 / efficiency_parameter) * ((vehicle_mass * acceleration + vehicle_mass * gravitational_constant * math.cos(angle_of_the_road) * ((1.75 / 1000) * (0.0328 * speed + 4.575)) + 0.5 * air_density * vehicle_frontal_area * aerodynamic_drag * speed ** 2 + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road)) * speed)
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    consumption = driving_consumption + sensor_consumption
                self.car['battery_capacity'] -= consumption
                consumption_during_tsp_float += consumption
                
                # Unloading
                yield self.env.timeout(random.uniform(service_time_min, service_time_max))
                
                
            # Append data in lists for the 'order' excel datasheet
            if create_excel == True:
                # reset lists for tsp to prevent repetitions 
                tspcar_id = []
                tspcar_origin = []
                distance_to_first_point = []
                time_to_first_point = []
                consumption_to_first_point = []
                distance_during_tsp = []
                time_of_tsp = []
                consumption_during_tsp = []
                tspcar_energy_consumed = []
                route_taken = []

            # Append data in lists for the 'tsp_out' excel datasheet
                tspcar_id.append(self.car['id'])
                tspcar_origin.append(car_origin_float)
                distance_to_first_point.append(distance_to_sart_tsp)
                time_to_first_point.append(time_diff_origin)
                consumption_to_first_point.append(consumption_to_first_point_float)
                distance_during_tsp.append(tour_distance)
                time_of_tsp.append(self.env.now - time_before_tour)
                self_car_battery_capacity = self.car['battery_capacity']
                #consumption_to_first_point0 = float(consumption_to_first_point[0])
                #print(f'Type of battery_before_tour: {type(battery_before_tour)}, Value: {battery_before_tour}')
                #print(f'Type of self_car_battery_capacity: {type(self_car_battery_capacity)}, Value: {self_car_battery_capacity}')
                #print(f'Type of consumption_to_first_point0: {type(consumption_to_first_point0)}, Value: {consumption_to_first_point0}')
                consumption_during_tsp.append(consumption_during_tsp_float)
                tspcar_energy_consumed.append(consumption_to_first_point_float + consumption_during_tsp_float)
                route_taken.append(tour_string)


            #if len(car_id) > 0:
                # Create a dictionary to store the data for the 'orders' sheet
                data_sheet_tsp_out = {
                    'Car ID': tspcar_id,
                    'Car Origin': tspcar_origin,
                    'Distance to Mailstation (m)': distance_to_first_point,
                    'Travel Time to Mailstation (s)': time_to_first_point,
                    'Consumption to Mailstation (Wh)': consumption_to_first_point,
                    'Distance of the TSP (m)': distance_during_tsp,
                    'Time of the TSP (s)': time_of_tsp,
                    'Consumption of the TSP (Wh)': consumption_during_tsp,
                    'Car Consumption (Wh)': tspcar_energy_consumed,
                    'TSP Route (Mailstation not includet)' : route_taken 
                }

                # Append the data to the list of data to save for the 'orders' sheet
                data_to_save_tsp_out.append(data_sheet_tsp_out)

                # Save the data to the Excel file
                save_data_to_excel(data_to_save_data, data_to_save_orders, data_to_save_tsp_out, data_to_save_tsp_in, data_to_save_charge, filename)
            
            # Count finished TSPs
            tsp_out_finished += 1
            
            if max_print_out == True:
                print(f"Car {self.car['id']} finished TSP morning tour after {self.env.now - time_before_tour}s and consumed {battery_before_tour - self.car['battery_capacity']} Wh after driving {tour_distance}m (distance to mailstation excluded)  Time: {self.env.now}")
            yield self.env.process(self.drive())
            
        # Define TSP time and ammount from Location to Mailstation
        if not tsp_in_per_day >= afternoon_tsp_ammount and afternoon_tsp_time <= self.env.now and self.car['battery_capacity'] > tsp_charging_threshold:
            tsp_in_per_day += 1
            locations_to_visit = list(tsp_locations)
            locations_to_visit.append(tsp_mailstation_location) # Append mailstation location to the list 
            car_origin_float_in = self.car['current_location']
            
            # Solve TSP with nearest neighbour
            if nearest_neighbour == True:
                tour = []
                unvisited_locations = locations_to_visit.copy()
                unvisited_locations.remove(tsp_mailstation_location) # remove mailstation location to the list 
                current_location = self.car['current_location']

                while len(unvisited_locations) > 0:
                    min_distance = float('inf')
                    nearest_location = None

                    nn_distance = 0
                    for location in unvisited_locations:
                        if use_a_star == True or use_dijkstra == True:
                            path, distance = a_star(current_location, location)
                            nn_distance = distance
                        if use_floyd_warshall == True:
                            distance = fw.get_shortest_distance(current_location, location)
                            nn_distance = distance

                        if nn_distance < min_distance:
                            min_distance = nn_distance
                            nearest_location = location

                    if nearest_location:
                        tour.append(nearest_location)
                        unvisited_locations.remove(nearest_location)
                        current_location = nearest_location
                    elif not unvisited_locations:
                        break  # Exit the loop when all locations are visited
                if self.car['current_location'] in tour:
                    tour.remove(self.car['current_location']) # Remove the Location because the car is already there
                tour.append(tsp_mailstation_location) # Append the mailstation as the last station
                tour_string = ', '.join(tour)  # Convert the tour to a comma-separated string
            
            # Solve TSP with brute force
            if brute_force:
                best_tour = None
                best_distance = float('inf')
                tour = []
                best_tour = []
                unvisited_locations = locations_to_visit.copy()
                #unvisited_locations.append(tsp_mailstation_location) # Append mailstation location to the list 
                print(f"({unvisited_locations}")
                current_location = self.car['current_location']

                def brute_force_tsp(tour, unvisited_locations):
                    nonlocal best_tour, best_distance
                    if not unvisited_locations:
                        # Calculate the total distance for the current tour
                        bf_distance = 0
                        for i in range(len(tour) - 1):
                            location1 = tour[i]
                            location2 = tour[i + 1]
                            if use_floyd_warshall == True:
                                distance = fw.get_shortest_distance(location1, location2)
                                bf_distance += distance
                            if use_a_star == True or use_dijkstra == True:
                                path, distance = a_star(location1, location2)
                                bf_distance += distance

                        # Check if it's a better tour
                        if bf_distance < best_distance and tour[-1] == tsp_mailstation_location:
                            best_distance = bf_distance
                            best_tour = tour

                    for location in unvisited_locations:
                        new_tour = tour + [location]
                        new_unvisited_locations = unvisited_locations.copy()
                        new_unvisited_locations.remove(location)
                        
                        # Calculate the current tour distance for this branch
                        branch_distance = 0
                        for i in range(len(new_tour) - 1):
                            location1 = new_tour[i]
                            location2 = new_tour[i + 1]
                            if use_floyd_warshall == True:
                                distance = fw.get_shortest_distance(location1, location2)
                                branch_distance += distance
                            if use_a_star == True or use_dijkstra == True:
                                path, distance = a_star(location1, location2)
                                branch_distance += distance

                        # Only continue the branch if the current branch distance is less than best_distance to save computational resources
                        if branch_distance < best_distance:
                            brute_force_tsp(new_tour, new_unvisited_locations)

                brute_force_tsp([current_location], locations_to_visit)
                if self.car['current_location'] in best_tour:
                    best_tour.remove(self.car['current_location']) # Remove the Location because the car is already there
                tour_string = ', '.join(best_tour)
            
            # Define path manually
            if pre_defined_path:
                current_location = self.car['current_location']
                
                if current_location in ['Location 1', 'Location 2', 'Location 4', 'Location 5', 'Location 6', 'Location 11', 'Location 12']:
                    tour_in = ['Location N', 'Location M', 'Location L', 'Location K', 'Location J', 'Location I', 'Location G', 'Location H', 'Location F', 'Location E', 'Location D', 'Location C', 'Location B', 'Location A', 'Location 1']
                
                if current_location in ['Location 7', 'Location 8', 'Location 9', 'Location 10']:
                    tour_in = ['Location K', 'Location J', 'Location I', 'Location G', 'Location H', 'Location N', 'Location M', 'Location L', 'Location F', 'Location E', 'Location D', 'Location C', 'Location B', 'Location A', 'Location 1']
                
                if current_location in ['Location 3']:
                    tour_in = ['Location D', 'Location C', 'Location N', 'Location M', 'Location L', 'Location K', 'Location J', 'Location I', 'Location G', 'Location H', 'Location F', 'Location E', 'Location B', 'Location A', 'Location 1']
                    
                if current_location in ['Location A', 'Location B', 'Location E', 'Location F', 'Location L', 'Location M', 'Location N']:
                    tour_in = ['Location N', 'Location M', 'Location L', 'Location K', 'Location J', 'Location I', 'Location G', 'Location H', 'Location F', 'Location E', 'Location D', 'Location C', 'Location B', 'Location A', 'Location 1']
                    tour_in.remove(current_location)
                    tour_in.insert(0, current_location)
                    
                if current_location in ['Location K', 'Location J', 'Location I', 'Location G', 'Location H']:
                    tour_in = ['Location K', 'Location J', 'Location I', 'Location G', 'Location H', 'Location N', 'Location M', 'Location L', 'Location F', 'Location E', 'Location D', 'Location C', 'Location B', 'Location A', 'Location 1']
                    tour_in.remove(current_location)
                    tour_in.insert(0, current_location)
                    
                if current_location in ['Location C', 'Location D']:
                    tour_in = ['Location D', 'Location C', 'Location N', 'Location M', 'Location L', 'Location K', 'Location J', 'Location I', 'Location G', 'Location H', 'Location F', 'Location E', 'Location B', 'Location A', 'Location 1']
                    tour_in.remove(current_location)
                    tour_in.insert(0, current_location)
                    
                tour_string = ', '.join(tour_in)

            if max_print_out == True:
                print(f"Car {self.car['id']} will do the afternoon tour (TSP): {tour_string} finishing at the mailstation.")  # Print the final tour as a string
            # Split tour_string into individual location names
            tour = tour_string.split(', ')
        
            # Drive the tour
            tour_distance = 0
            battery_before_tour = self.car['battery_capacity']
            time_before_tour = self.env.now
            for next_location in tour:
                if use_a_star == True or use_dijkstra == True:
                    path, distance = a_star(self.car['current_location'], next_location)
                if use_floyd_warshall == True:
                    path = fw.get_shortest_path(self.car['current_location'], next_location)
                    distance = fw.get_shortest_distance(self.car['current_location'], next_location)
                distance_to_location = distance * (1+ (random.uniform(additional_distance_min, additional_distance_max)))      

                drive_to_location = self.env.now
                if max_print_out == True:
                    print(f"Car {self.car['id']} drives {distance_to_location}m from {self.car['current_location']} to {next_location} as part of its TSP tour. Time: {self.env.now}")
                yield self.env.timeout((distance_to_location / speed) * (1+ (random.uniform(additional_time_min, additional_time_max))))
                tour_distance += distance_to_location
                self.car['current_location'] = next_location
                arrive_at_location = self.env.now
                time_diff = arrive_at_location - drive_to_location

                if use_fixed_consumption == True:
                    consumption = time_diff * self.car['sensor_consumption_rate'] + distance_to_location * self.car['battery_consumption_rate']
                if use_formula_shao == True: 
                    driving_consumption = ((distance_to_location / speed) / 3600) * ((speed / efficiency_parameter) * (vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 2) + vehicle_mass * mass_factor * acceleration))
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    consumption = driving_consumption + sensor_consumption
                if use_formula_baktas == True:
                    driving_consumption = ((distance_to_location / speed) / 3600) * (1 / efficiency_parameter) * ((vehicle_mass * acceleration * speed + vehicle_mass * gravitational_constant * speed * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 3) + vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) * speed))
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    consumption = driving_consumption + sensor_consumption
                if use_formula_fiori == True:
                    driving_consumption = ((distance_to_location / speed) / 3600) * (1 / efficiency_parameter) * ((vehicle_mass * acceleration + vehicle_mass * gravitational_constant * math.cos(angle_of_the_road) * ((1.75 / 1000) * (0.0328 * speed + 4.575)) + 0.5 * air_density * vehicle_frontal_area * aerodynamic_drag * speed ** 2 + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road)) * speed)
                    sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                    consumption = driving_consumption + sensor_consumption
                self.car['battery_capacity'] -= consumption
                # Unloading
                yield self.env.timeout(random.uniform(service_time_min, service_time_max))
                
            # Append data in lists for the 'order' excel datasheet
            if create_excel == True:
                # reset lists for tsp to prevent repetitions 
                tspcar_id = []
                tspcar_origin = []
                distance_to_first_point = []
                #time_to_first_point = []
                #consumption_to_first_point = []
                distance_during_tsp = []
                time_of_tsp = []
                consumption_during_tsp = []
                tspcar_energy_consumed = []
                route_taken = []

            # Append data in lists for the 'tsp_out' excel datasheet
                tspcar_id.append(self.car['id'])
                tspcar_origin.append(car_origin_float_in)
                if tour_string[:10] == car_origin_float_in:
                    distance_to_first_point.append(f'Yes')
                else:
                    distance_to_first_point.append(f'No: {car_origin_float_in}')
                #time_to_first_point.append(time_diff_origin)
                #consumption_to_first_point.append(consumption_to_first_point)
                distance_during_tsp.append(tour_distance)
                time_of_tsp.append(self.env.now - time_before_tour)
                consumption_during_tsp.append(battery_before_tour - self.car['battery_capacity'])
                #tspcar_energy_consumed.append(battery_before_tour - self.car['battery_capacity'])
                route_taken.append(tour_string)


                # Create a dictionary to store the data for the 'orders' sheet
                data_sheet_tsp_in = {
                    'Car ID': tspcar_id,
                    'Car Origin': tspcar_origin,
                    'Origin = first tour point?': distance_to_first_point,
                    #'Travel Time to First Point (s)': time_to_first_point,
                    #'Consumption to First Point (Wh)': consumption_to_first_point,
                    'Distance of the TSP (m)': distance_during_tsp,
                    'Time of the TSP (s)': time_of_tsp,
                    'Consumption of the TSP (Wh)': consumption_during_tsp,
                    #'Car Consumption (s)': tspcar_energy_consumed,
                    'TSP Route' : route_taken 
                }

                # Append the data to the list of data to save for the 'orders' sheet
                data_to_save_tsp_in.append(data_sheet_tsp_in)

                # Save the data to the Excel file
                save_data_to_excel(data_to_save_data, data_to_save_orders, data_to_save_tsp_out, data_to_save_tsp_in, data_to_save_charge, filename)
            
            # Count finidhes TSPs
            tsp_in_finished += 1
            
            if max_print_out == True:
                print(f"Car {self.car['id']} finished TSP afternoon tour after {self.env.now - time_before_tour}s and consumed {battery_before_tour - self.car['battery_capacity']} Wh after driving {tour_distance}m (distance to mailstation included)  Time: {self.env.now}")
                yield self.env.process(self.drive())    
        yield self.env.process(self.drive())
            
            
    def search_nearest_station(self):
        # Declare global variables
        global total_cars_started_charging
        global max_print_out
        # Initialize variables to store the nearest station's location and ID
        nearest_station_location = None
        nearest_station_id = None
        min_distance = float('inf')  # Initialize with infinity
        self.charge_car_origin = self.car['current_location']

        # Iterate through charging stations to find the nearest available station with no cars
        for location, stations in charging_stations.items():
            for station in stations:
                if station['available'] == True and station['queue_length'] == 0:
                    # Calculate the distance from the car's current location to the charging station using A*
                    if use_a_star == True or use_dijkstra == True:
                        path, distance = a_star(self.car['current_location'], location)
                    if use_floyd_warshall == True:
                        path = fw.get_shortest_path(self.car['current_location'], location)
                        distance = fw.get_shortest_distance(self.car['current_location'], location)
                    if max_print_out == True:
                        print(f" calculating Station {station['location']} - ID {station['id']}. nearest")

                    # Update the nearest station if this one is closer
                    if distance < min_distance:
                        min_distance = distance
                        nearest_station_location = location
                        nearest_station_id = station['id']

        # Set the car's nearest station location and ID
        self.charging_station_location = nearest_station_location
        self.charging_station_id = nearest_station_id

        if nearest_station_location is not None and nearest_station_id is not None:
            station = charging_stations[self.charging_station_location][self.charging_station_id - 1]
            station['available'] = False  # Mark the station as unavailable
            self.first_of_the_queue = True # Mark self as first to skip the "Wait until the station is actually free"-check
            if max_print_out == True:
                print(f"Test nearest (queue length should be 0 here): Station {station['location']} - ID {station['id']} - Queue Length: {station['queue_length']}")
            station['queue_length'] += 1
            total_cars_started_charging += 1
            if max_print_out == True:
                print(f"Test nearest: Station {station['location']} - ID {station['id']} - Queue Length: {station['queue_length']}")
            self.charger_reservation_time = self.env.now
            if max_print_out == True:
                print(f"Car {self.car['id']} found nearest charging station at {nearest_station_location}, station ID: {nearest_station_id}")

            
            # Set an predicted start charging timestep to allow the calculation of waiting_time (it will get updatd later)
            time_now = self.env.now
            predicted_started_charging_timestamp = time_now + min_distance / speed
            station['started_charging_timestamp'].clear()
            station['finished_charging_timestamp'].clear()
            station['started_charging_timestamp'].append(predicted_started_charging_timestamp)
                   
            # Drive to the nearest charging station and track battery consumption
            drive_to_charger_start_time = self.env.now
            drive_distance_to_charger = min_distance * (1+ (random.uniform(additional_distance_min, additional_distance_max)))                                                            
            yield self.env.timeout((drive_distance_to_charger / speed) * (1+ (random.uniform(additional_time_min, additional_time_max))))
            self.car['current_location'] = self.charging_station_location
            drive_to_charger_arrive_time = self.env.now
            time_diff = drive_to_charger_arrive_time - drive_to_charger_start_time
            if use_fixed_consumption == True:
                consumption = time_diff * self.car['sensor_consumption_rate'] + drive_distance_to_charger * self.car['battery_consumption_rate']
            if use_formula_shao == True: 
                driving_consumption = ((drive_distance_to_charger / speed) / 3600) * ((speed / efficiency_parameter) * (vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 2) + vehicle_mass * mass_factor * acceleration))
                sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                consumption = driving_consumption + sensor_consumption
            if use_formula_baktas == True:
                driving_consumption = ((drive_distance_to_charger / speed) / 3600) * ((vehicle_mass * acceleration * speed + vehicle_mass * gravitational_constant * speed * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 3) + vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) * speed))
                sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                consumption = driving_consumption + sensor_consumption
            if use_formula_fiori == True:
                driving_consumption = ((drive_distance_to_charger / speed) / 3600) * ((vehicle_mass * acceleration + vehicle_mass * gravitational_constant * math.cos(angle_of_the_road) * ((1.75 / 1000) * (0.0328 * speed + 4.575)) + 0.5 * air_density * vehicle_frontal_area * aerodynamic_drag * speed ** 2 + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road)) * speed)
                sensor_consumption = time_diff * self.car['sensor_consumption_rate']
                consumption = driving_consumption + sensor_consumption
            self.car['battery_capacity'] -= consumption
            self.consumption_to_station = consumption
            self.distance_to_station = drive_distance_to_charger
            self.travel_time_to_Station = time_diff
            
            # Set timestamp for arrival at charging station
            self.arrived_at_charger_time = self.env.now
            if max_print_out == True:
                print(f"Car {self.car['id']} arrived at the nearest available Station {station['location']} - ID {station['id']} at Time: {self.env.now}. Consuming {consumption} Wh on the way to the charging station.")

            yield self.env.process(self.charge())  # Yield to the charge method

        else:
            # No available charging station found, yield to the shortest_queue_station method
            yield self.env.process(self.shortest_queue_station())

            
    def shortest_queue_station(self):
        # Declare global variables
        global total_cars_started_charging
        global max_print_out
        # Initialize variables to track the chosen station and waiting time
        chosen_station_location = None
        chosen_station_id = None
        min_waiting_time = float('inf')  # Initialize with a large value
        time_now = self.env.now

        # Iterate through charging stations to find the one with the shortest queue
        for location, stations in charging_stations.items():
            for station in stations:
                # Calculate the waiting time for the current station
                num_cars_in_queue = station['queue_length']
                
                # If the previous car just left, work with the finished_charging_timestamp
                if 0 == len(station['started_charging_timestamp']) and 1 == len(station['finished_charging_timestamp']):
                    waiting_time = (num_cars_in_queue * charging_time)
                    if max_print_out == True:
                        print(f" used finished timestamp. Car {self.car['id']} Station {station['location']} - ID {station['id']}")
                else:
                    if 1 == len(station['started_charging_timestamp']) and 0 == len(station['finished_charging_timestamp']):
                        oldest_timestamp = min(station['started_charging_timestamp'])
                        if max_print_out == True:
                            print(f" Number of cars in queue: {num_cars_in_queue}. time now: {time_now}. oldest timestamp: {oldest_timestamp} in Queue Length: {station['queue_length']} for Station {station['location']} - ID {station['id']}")
                        waiting_time = ((num_cars_in_queue - 1) * charging_time) + (charging_time - (time_now - oldest_timestamp))
                        if max_print_out == True:
                            print(f"used start timestamp. Car {self.car['id']} Station {station['location']} - ID {station['id']}")
                    else:
                        print(f" error in shortest queue station logic. start timestamp:{station['started_charging_timestamp']}, finish timestamp {station['finished_charging_timestamp']}") 
                        print(f" Station: {station['location']} - ID {station['id']}")
                # Check if this station has the shortest waiting time so far
                if waiting_time < min_waiting_time:
                    min_waiting_time = waiting_time
                    chosen_station_location = location
                    chosen_station_id = station['id']
        time_now = None

        # Set the car's nearest station location and ID
        self.charging_station_location = chosen_station_location
        self.charging_station_id = chosen_station_id
        station = charging_stations[self.charging_station_location][self.charging_station_id - 1] # Python List starts at 0
        
        # Set timestamp and add car to queue and total_cars_started_charging
        self.charger_reservation_time = self.env.now  
        if max_print_out == True:
            print(f"Test queue3: Station {station['location']} - ID {station['id']} - Queue Length: {station['queue_length']}")
        station['queue_length'] += 1
        total_cars_started_charging += 1
        if max_print_out == True:
            print(f"Test queue4 (should be +1): Station {station['location']} - ID {station['id']} - Queue Length: {station['queue_length']}")

        # Drive to chosen station and track battery consumption
        drive_to_charger_start_time = self.env.now
        if use_a_star == True or use_dijkstra == True:
            path, distance = a_star(self.car['current_location'], chosen_station_location)
        if use_floyd_warshall == True:
            path = fw.get_shortest_path(self.car['current_location'], chosen_station_location)
            distance = fw.get_shortest_distance(self.car['current_location'], chosen_station_location)
        drive_distance_to_charger = distance * (1+ (random.uniform(additional_distance_min, additional_distance_max))) 
        yield self.env.timeout((drive_distance_to_charger / speed) * (1+ (random.uniform(additional_time_min, additional_time_max)))) 
        self.car['current_location'] = self.charging_station_location
        drive_to_charger_arrive_time = self.env.now
        time_diff = drive_to_charger_arrive_time - drive_to_charger_start_time
        if use_fixed_consumption == True:
            consumption = time_diff * self.car['sensor_consumption_rate'] + drive_distance_to_charger * self.car['battery_consumption_rate']
        if use_formula_shao == True: 
            driving_consumption = ((drive_distance_to_charger / speed) / 3600) * ((speed / efficiency_parameter) * (vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 2) + vehicle_mass * mass_factor * acceleration))
            sensor_consumption = time_diff * self.car['sensor_consumption_rate']
            consumption = driving_consumption + sensor_consumption
        if use_formula_baktas == True:
            driving_consumption = ((drive_distance_to_charger / speed) / 3600) * (1 / efficiency_parameter) * ((vehicle_mass * acceleration * speed + vehicle_mass * gravitational_constant * speed * math.sin(angle_of_the_road) + 0.5 * vehicle_frontal_area * air_density * aerodynamic_drag * (speed ** 3) + vehicle_mass * gravitational_constant * rolling_resistance * math.cos(angle_of_the_road) * speed))
            sensor_consumption = time_diff * self.car['sensor_consumption_rate']
            consumption = driving_consumption + sensor_consumption
        if use_formula_fiori == True:
            driving_consumption = ((drive_distance_to_charger / speed) / 3600) * (1 / efficiency_parameter) * ((vehicle_mass * acceleration + vehicle_mass * gravitational_constant * math.cos(angle_of_the_road) * ((1.75 / 1000) * (0.0328 * speed + 4.575)) + 0.5 * air_density * vehicle_frontal_area * aerodynamic_drag * speed ** 2 + vehicle_mass * gravitational_constant * math.sin(angle_of_the_road)) * speed)
            sensor_consumption = time_diff * self.car['sensor_consumption_rate']
            consumption = driving_consumption + sensor_consumption
        self.car['battery_capacity'] -= consumption
        self.consumption_to_station = consumption
        self.distance_to_station = drive_distance_to_charger
        self.travel_time_to_Station = time_diff

        # Set timestamp for arrival at charging station
        self.arrived_at_charger_time = self.env.now
        if max_print_out == True:
            print(f"Car {self.car['id']} arrived at Station with the shortest queue {station['location']} - ID {station['id']} at Time: {self.env.now}. Consuming {consumption} Wh on the wy to the charging station.")

        # Queue up at the chosen station
        if chosen_station_location is not None and chosen_station_id is not None:
            station = charging_stations[self.charging_station_location][self.charging_station_id - 1] # Python List starts at 0
            station['cars'].append(self.car['id'])
            station['reserved_charger_timestamp'].append(self.charger_reservation_time)
            station['arrived_at_charger_timestamp'].append(self.arrived_at_charger_time)

            # Calculate the timeout minus the travel
            timeout_duration = min_waiting_time - (distance / speed)
            
            if max_print_out == True:
                print(f" Car {self.car['id']} will wait for {timeout_duration} in queue for Station {station['location']} - ID {station['id']} start timestamp:{station['started_charging_timestamp']}, finish timestamp {station['finished_charging_timestamp']}. Time: {self.env.now}")
            # Yield for the calculated timeout duration
            if timeout_duration > 0:
                yield self.env.timeout(timeout_duration)
            #if timeout_duration < 0:
                #print(f" Car {self.car['id']} would be: {timeout_duration}") # delete
            if max_print_out == True:
                print(f" Car {self.car['id']} waited for {timeout_duration} in queue for Station {station['location']} - ID {station['id']}. Time: {self.env.now}")

            
            # Start charging
            yield self.env.process(self.charge())

        else:
            print(f"Car {self.car['id']} Can not be recharged (no charging stations exists or logic error)")


        
    def charge(self):
        # Declare global variables
        global max_print_out
        global total_waiting_time_till_charge  
        global total_charged_ammount
        global total_cars_finished_charging
        global negative_battery
        # Get the charging station information
        station = charging_stations[self.charging_station_location][self.charging_station_id - 1] # Python List starts at 0
        # Check if the car has a reserved charger and if its available
        if self.charging_station_location is not None and self.charging_station_id is not None and self.car['in_charging'] == False:
            # Wait until the station is actually free (delay due to service time and driving time)
            if max_print_out == True:
                print(f"Car {self.car['id']} self.first_of_the_queue : {self.first_of_the_queue}")
            if station['queue_length'] == 0:
                print(f"Car {self.car['id']} Queue length at at Station {station['location']} - ID {station['id']} is 0. Error in logic")
            if self.first_of_the_queue == False:
                timeout_counter = 0 # For error prevention
                while len(station['finished_charging_timestamp']) == 0:
                    yield self.env.timeout(1)
                    timeout_counter += 1
                    if timeout_counter > 3600 * 2:
                        print(f"Error: Car {self.car['id']} at Station {station['location']} - ID {station['id']} trapped in loop: {timeout_counter}")
                        yield self.env.timeout(1000)
                        timeout_counter += 1000
            
            # Make sure that station unavailable
            station['available'] = False
            
            # Check if the car arrived with negative battery capacity (if yes the charging_threshhold is set too low)
            if self.car['battery_capacity'] < 0:
                negative_battery += 1

            # Check if the car's ID is not in the list (this is the case when the car comes from shortest_queue_station)
            if self.car['id'] not in station['cars']:
                # If it's not append timestamps 
                station['cars'].append(self.car['id'])
                station['reserved_charger_timestamp'].append(self.charger_reservation_time)
                station['arrived_at_charger_timestamp'].append(self.arrived_at_charger_time)
            
            # Append the start charging time to list and clear finish charging timestamp list and initial start timestamp
            station['started_charging_timestamp'].clear()
            self.charging_start_time = self.env.now
            station['finished_charging_timestamp'].clear()
            station['started_charging_timestamp'].append(self.charging_start_time)
            if max_print_out == True:
                print(f"Appended start time {self.charging_start_time} Car: {self.car['id']} at Station {station['location']} - ID {station['id']} - Queue Length: {station['queue_length']}")

            # Calculate and add up the waiting time until the car starts charging 
            waiting_time_till_charge = self.charging_start_time - self.arrived_at_charger_time
            total_waiting_time_till_charge += waiting_time_till_charge

            # Simulate service time 
            yield self.env.timeout(random.uniform(service_time_min, service_time_max))
            self.car['in_charging'] = True
            
            # Append data in lists for the 'charge' excel datasheet
            if create_excel == True:
                # reset lists for charging to prevent repetitions 
                charge_car_id = []
                charge_car_origin = []
                carging_station_location = []
                charging_station_id = []
                consumption_to_station = []
                distance_to_station = []
                travel_time_to_station = []
                arrived_with_capacity = []
                waiting_time_at_station = []

                # Append data in lists for the 'charge' excel datasheet
                charge_car_id.append(self.car['id'])
                charge_car_origin.append(self.charge_car_origin)
                carging_station_location.append(self.charging_station_location)
                charging_station_id.append(self.charging_station_id)
                consumption_to_station.append(self.consumption_to_station)
                distance_to_station.append(self.distance_to_station)
                travel_time_to_station.append(self.travel_time_to_Station)
                arrived_with_capacity.append(self.car['battery_capacity'])
                waiting_time_at_station.append(self.charging_start_time - self.arrived_at_charger_time)


                # Create a dictionary to store the data for the 'orders' sheet
                data_sheet_charge = {
                    'Car ID': charge_car_id,
                    'Car Origin': charge_car_origin,
                    'Station Location': carging_station_location,
                    'Station ID': charging_station_id,
                    'Consumption to Station (Wh)': consumption_to_station,
                    'Distance to Station (m)': distance_to_station,
                    'Time to Station (s)': travel_time_to_station,
                    'Capacity left (Wh)': arrived_with_capacity,
                    'Waitingtime from arrival till start cahrging (s)': waiting_time_at_station
                }

                # Append the data to the list of data to save for the 'orders' sheet
                data_to_save_charge.append(data_sheet_charge)

                # Save the data to the Excel file
                save_data_to_excel(data_to_save_data, data_to_save_orders, data_to_save_tsp_out, data_to_save_tsp_in, data_to_save_charge, filename)
            
            
            # Yield timeout for charging_time and track charged electricity ammount and number of cars recharged
            if max_print_out == True:
                print(f"Car {self.car['id']} started charging at {self.charging_station_location}, station ID: {self.charging_station_id}. It arrived with battery capacity reserve of: {self.car['battery_capacity']} Wh. Time now: {self.env.now}. self charging start time: {self.charging_start_time} self arrived at cahrger time: {self.arrived_at_charger_time}")
            yield self.env.timeout(charging_time)
            battery_capacity_pre_charge = self.car['battery_capacity']
            self.car['battery_capacity'] = battery_capacity 
            total_charged_ammount += battery_capacity - battery_capacity_pre_charge
            
            # Simulate service time 
            yield self.env.timeout(random.uniform(service_time_min, service_time_max))
            total_cars_finished_charging += 1
            
            # Appened finished charging timestamp after clearing started charging list
            station['started_charging_timestamp'].clear()
            station['finished_charging_timestamp'].clear()
            self.charging_finish_time = self.env.now
            station['finished_charging_timestamp'].append(self.charging_finish_time)
            

            # Charging is complete, remove the car from the charger's list and reset timestamps
            station['cars'].remove(self.car['id'])
            station['reserved_charger_timestamp'].remove(self.charger_reservation_time)
            station['arrived_at_charger_timestamp'].remove(self.arrived_at_charger_time)
            if max_print_out == True:
                print(f"Car {self.car['id']} finished charging at {self.charging_station_location}, station ID: {self.charging_station_id}. Idle waiting time was {waiting_time_till_charge}. Current battery capacity: {self.car['battery_capacity']}. Time now: {self.env.now}")
            self.first_of_the_queue = False
            self.charging_start_time = None 
            self.charger_reservation_time = None
            self.arrived_at_charger_time = None
            self.charging_finish_time = None
            self.charging_start_time = None  

            # Yield to the drive process after adjusting availibility of the car and station
            self.car['available'] = True
            self.car['in_charging'] = False
            station['queue_length'] -= 1
            # Mark the station as available for the search_nearest_station Method if queue is 0
            if station['queue_length'] == 0:
                station['available'] = True
            yield self.env.process(self.drive())
        else:
            print(f"Car {self.car['id']} Can not be recharged. Error in charge logic")


def print_charging_station_status(env, charging_stations, print_charging_queue):
    global max_print_out
    while True:
        if max_print_out == True:
            yield env.timeout(print_charging_queue)  # Wait for the specified print_charging_queue interval
            print("\nCharging Station Availability and Car Lists:")
            for location, stations in charging_stations.items():
                print(f"Location: {location}")
                for station in stations:
                    availability = "Available" if station['available'] == True else "Occupied"
                    car_list = ", ".join([f"Car {car_id}" for car_id in station['cars']])
                    print(f"Station ID: {station['id']} - Status: {availability} - Cars: {car_list}")

                    # Print the list of started charging timestamps
                    started_charging_list = ", ".join([f"Car {car_id} ({timestamp})" for car_id, timestamp in zip(station['cars'], station['started_charging_timestamp'])])
                    finished_charging_timestamp = ", ".join([f"Car {car_id} ({timestamp})" for car_id, timestamp in zip(station['cars'], station['finished_charging_timestamp'])])
                    if len(station['finished_charging_timestamp']) == 0:
                        print(f"  Started Charging: {started_charging_list}")
                    else:    
                        print(f"  Finished Charging: {finished_charging_timestamp}")    

                    # Print the list of arrival at charger timestamps
                    arrived_at_charger_list = ", ".join([f"Car {car_id} ({timestamp})" for car_id, timestamp in zip(station['cars'], station['arrived_at_charger_timestamp'])])
                    print(f"  Arrived Charger:  {arrived_at_charger_list}")

                    # Print the list of reserved charger timestamps
                    reserved_charger_list = ", ".join([f"Car {car_id} ({timestamp})" for car_id, timestamp in zip(station['cars'], station['reserved_charger_timestamp'])])
                    print(f"  Reserved Charger: {reserved_charger_list}")

# Function to save data to an Excel file
if create_excel == True:
    def save_data_to_excel(data, orders_data, tsp_out_data, tsp_in_data, charge_data, filename):
        with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
            df_data = pd.DataFrame(data)
            df_orders = pd.DataFrame(orders_data)
            df_tspout = pd.DataFrame(tsp_out_data)
            df_tspin = pd.DataFrame(tsp_in_data)
            df_charge = pd.DataFrame(charge_data)
            df_data.to_excel(writer, sheet_name='data', index=False)
            df_orders.to_excel(writer, sheet_name='orders', index=False)
            df_tspout.to_excel(writer, sheet_name='tsp_out', index=False)
            df_tspin.to_excel(writer, sheet_name='tsp_in', index=False)
            df_charge.to_excel(writer, sheet_name='charge', index=False)
    
# Generate a unique filename with a timestamp in global
if create_excel == True:
    timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"Simulation_{timestamp}.xlsx"

# Create a list to store simulation data
if create_excel == True:
    data_to_save_data = []    # Data for 'data' sheet
    data_to_save_orders = []  # Data for 'orders' sheet
    data_to_save_tsp_out = []
    data_to_save_tsp_in = []
    data_to_save_charge = []
    
# Create the process to save data in excel
if create_excel:
    def save_in_excel(env, save_every, data_to_save_data, data_to_save_orders, data_to_save_tsp_out, data_to_save_tsp_in, data_to_save_charge, filename):
        global num_deliveries_finished
        global total_orders
        global total_waiting_time_till_charge
        global total_charged_ammount
        global total_cars_started_charging
        global total_cars_finished_charging
        # For Excel
        global order_nr
        global additional_order_nr
        global order_time
        global additional_order_time
        global order_weight
        global additional_order_weight
        global total_order_weight
        global order_pickup_time
        global order_delivery_time
        global car_id
        global car_origin
        global order_origin
        global order_destination
        global order_energy_consumed
        global car_energy_consumed
        global order_distance_traveled
        global car_distance_traveled
        global car_pickupt_distance
        global waiting_for_order
        # globals for tsp Excel
        global tspcar_id
        global tspcar_origin
        global distance_to_first_point
        global time_to_first_poi
        global consumption_to_first_point
        global distance_during_tsp
        global time_of_tsp
        global consumption_during_tsp
        global tspcar_energy_consumed
        global route_taken
        # lists for charging excel
        global charge_car_id 
        global charge_car_origin 
        global carging_station_location 
        global charging_station_id
        global consumption_to_station 
        global distance_to_station 
        global travel_time_to_station 
        global arrived_with_capacity 
        global waiting_time_at_station 

        while True:   
            yield env.timeout(save_every)  # Save every x minute             
            if max_print_out == True:
                print(f"Data is being saved to excel now (Time:{env.now})")

            # Create a dictionary to store the data
            data_sheet_data = {
                'Simulation Time': env.now,
                'Deliveries Finished': num_deliveries_finished,
                'Total Orders': total_orders,
                'Started morning TSP': tsp_out_per_day,
                'Started afternoon TSP': tsp_in_per_day,
                'Total Waiting Time till Charge': total_waiting_time_till_charge,
                'Total Charged Amount': total_charged_ammount,
                'Total Cars Started Charging': total_cars_started_charging,
                'Total Cars Finished Charging': total_cars_finished_charging
            }

            # Append the data to the list of data to save for the 'data' sheet
            data_to_save_data.append(data_sheet_data)
            
            # Save the data to the Excel file
            save_data_to_excel(data_to_save_data, data_to_save_orders, data_to_save_tsp_out, data_to_save_tsp_in, data_to_save_charge, filename)           
     
        
def make_remaining_cars_available(env, cars):
    global initially_available
    global fleet_size
    global second_group
    yield env.timeout(second_group)
    for i in range(initially_available, fleet_size):
        cars[i]['available'] = True
        if max_print_out == True:
            print(f"Car {cars[i]['id']} is now available")
    if max_print_out == True:        
        print(f"Second group was released. Time: {env.now}")


                
# Function to run the simulation and collect the number of unfinished orders
def run_simulation(env, simulation_num):
    # Declare globals
    global all_simulations_total_orders
    global num_unfinished_orders
    global global_deliveries_finished
    global num_deliveries_finished
    global total_orders
    global num_deliveries_started
    global total_waiting_time_till_charge
    global all_simulations_total_waiting_time_till_charge
    global total_charged_ammount
    global all_simulation_total_charged_ammount
    global total_cars_started_charging 
    global all_simulations_total_cars_started_charging 
    global total_cars_finished_charging
    global all_simulations_total_cars_finished_charging
    global initially_available
    global negative_battery
    global all_simulations_negative_battery
    global tsp_out_per_day
    global tsp_in_per_day
    global tsp_out_finished
    global tsp_in_finished

    if use_floyd_warshall == True and simulation_num == 1:
        # Run the Floyd-Warshall algorithm to compute all shortest distances
        fw.run_floyd_warshall()
    
    # Create an order queue
    order_queue = simpy.Store(env)

    # Create cars in the fleet
    cars = []
    for i in range(fleet_size):
        # Randomly select a starting location based on probabilities
        starting_location = random.choices(
            list(starting_location_probabilities.keys()),
            weights=list(starting_location_probabilities.values())
        )[0]
        
        # Define what fraction of the cars is initially available 
        initially_available_cars = True if 0 <= i < initially_available else False  
        
        # Initial states of the cars
        car = {
            'id': i + 1,
            'available': initially_available_cars,
            'in_charging': False,
            'current_location': starting_location,
            'travel_time': 0,
            'battery_capacity': battery_capacity, 
            'battery_consumption_rate': battery_consumption_rate,
            'sensor_consumption_rate': sensor_consumption_rate,
            'total_travel_distance': 0, # wird bis jetzt noch nicht benutzt
            'total_travel_time': 0, # wird bis jetzt noch nicht benutzt
            'current_package_weight': 0
        }
        cars.append(car)

    # Start the order generation process
    env.process(generate_order(env, order_queue))

    # Start the car processes
    for car in cars:
        car_process_variable = CarProcess(env, car, order_queue, charging_stations)
    
    # Release the second group of cars
    env.process(make_remaining_cars_available(env, cars))
    
    # Start the process to periodically print charging station status
    if max_print_out == True:
        env.process(print_charging_station_status(env, charging_stations, print_charging_queue=60*30))
    
    # Start the process to periodically save data to Excel
    if create_excel == True:
        env.process(save_in_excel(env, save_every, data_to_save_data, data_to_save_orders, data_to_save_tsp_out, data_to_save_tsp_in, data_to_save_charge, filename))

    # Run the simulation
    env.run(until=simulation_time)
    
    # Clear the 'cars', 'started_charging_timestamp', and 'reserved_charger_timestamp' lists and set 'available' to True for all charging stations
    for location, stations in charging_stations.items():
        for station in stations:
            station['cars'] = []
            station['started_charging_timestamp'] = [0]
            station['finished_charging_timestamp'] = []
            station['reserved_charger_timestamp'] = []
            station['arrived_at_charger_timestamp'] = []
            station['queue_length']= 0 
            station['available'] = True

    # Check if there are any unfinished orders
    unfinished_orders = total_orders - num_deliveries_finished
    num_unfinished_orders += unfinished_orders
    unfinished_order_ids = [f"Order {order['id']}" for order in order_queue.items] # These are untouched orders

    # Print simulation results
    print(f"\nSimulation {simulation_num} results:")
    print(f"Number of Finished Deliveries: {num_deliveries_finished}")
    print(f"Total Orders: {total_orders}")
    print(f"Total waiting time of all cars until recharge: {total_waiting_time_till_charge}")
    print(f"Total recharged ammount of all cars in Wh: {total_charged_ammount}")
    print(f"Total charging requests: {total_cars_started_charging}")
    print(f"Total charging completions: {total_cars_finished_charging}")
    print(f"{negative_battery} cars had not enough battery capcity")
    
    # Save the simulation results globally across all simulations
    all_simulations_total_orders += total_orders
    global_deliveries_finished += num_deliveries_finished
    all_simulations_total_waiting_time_till_charge += total_waiting_time_till_charge
    all_simulation_total_charged_ammount += total_charged_ammount
    all_simulations_total_cars_started_charging += total_cars_started_charging
    all_simulations_total_cars_finished_charging += total_cars_finished_charging
    all_simulations_negative_battery += negative_battery

    if unfinished_orders > 0:
        print(f"{unfinished_orders} unfinished orders and untouched orders: ", end="")
        print(", ".join(unfinished_order_ids))
        print(f"\n\n")
    else:
        print("All orders have been delivered within the time limit.\n\n")

    return unfinished_order_ids



# Run the simulation multiple times and calculate the average number of unfinished orders
num_simulations = number_of_simulations
unfinished_orders_list = []

for i in range(num_simulations):
    # Initialize the simulation environment for each iteration
    env = simpy.Environment()
    num_deliveries_finished = 0
    total_orders = 0
    num_deliveries_started = 0
    total_waiting_time_till_charge = 0
    total_charged_ammount = 0
    total_cars_started_charging = 0
    total_cars_finished_charging = 0
    negative_battery = 0
    tsp_out_per_day = 0
    tsp_in_per_day = 0
    unfinished_order_ids = run_simulation(env, i + 1)
    unfinished_orders_list.append(unfinished_order_ids)
    if create_excel == True:
        # Lists for tsp 
        tspcar_id = []
        tspcar_origin = []
        distance_to_first_point = []
        time_to_first_point = []
        consumption_to_first_point = []
        distance_during_tsp = []
        time_of_tsp = []
        consumption_during_tsp = []
        tspcar_energy_consumed = []
        route_taken = []
        # lists for charging excel
        charge_car_id = []
        charge_car_origin = []
        carging_station_location = []
        charging_station_id = []
        consumption_to_station = []
        distance_to_station = []
        travel_time_to_station = []
        arrived_with_capacity = []
        waiting_time_at_station = []

# Calculate the average number of unfinished and finished and total orders
average_total_orders = all_simulations_total_orders / num_simulations
average_unfinished_orders = num_unfinished_orders / num_simulations
avergage_global_deliveries_finished = global_deliveries_finished / num_simulations

# Calculate the average total waiting time of all cars over all simulations until recharge
average_total_waiting_time_till_charge = all_simulations_total_waiting_time_till_charge / num_simulations
average_total_waiting_time_till_charge_minutes_per_car = average_total_waiting_time_till_charge / 60 / fleet_size

# Calculate the average total recharged ammount of all cars over all simulations
average_total_charged_ammount = all_simulation_total_charged_ammount / num_simulations
average_total_charged_ammount_per_car = average_total_charged_ammount / fleet_size

# Calculate the average total ammount of charging requests 
average_total_cars_started_charging = all_simulations_total_cars_started_charging / num_simulations
average_charging_requests_per_car = average_total_cars_started_charging / fleet_size

# Calculate the average total ammount of charging completions
average_total_cars_finished_charging = all_simulations_total_cars_finished_charging / num_simulations
average_charging_completions_per_car = average_total_cars_finished_charging / fleet_size

# Calculate the average ammount of cars per simulation which had a negativa battery capacity
avergae_total_negative_battery = all_simulations_negative_battery / num_simulations

print(f"\n\033[1mAverage Results over all Simulations:\033[0m") # \033[1m text here will be fat \033[0m
print(f"\nAverage Number of total Orders per Simulation: {average_total_orders}")
print(f"\nAverage Number of unfinished Orders: {average_unfinished_orders}")
print(f"\nAverage Number of finished Orders: {avergage_global_deliveries_finished}")
print(f"\nAverage Number of finished morning TSPs: {tsp_out_finished / num_simulations}")
print(f"\nAverage Number of finished afternoon TSPs: {tsp_in_finished / num_simulations}")
print(f"\nAverage total waiting time of all cars until recharge: {average_total_waiting_time_till_charge} ({average_total_waiting_time_till_charge_minutes_per_car} minutes per car)")
print(f"\nAverage total recharged ammount of all cars in Wh: {average_total_charged_ammount} ({average_total_charged_ammount_per_car} Wh per car)")
print(f"\nAverage ammount of charging requests: {average_total_cars_started_charging} ({average_charging_requests_per_car} per car).")
print(f"\nAverage ammount of charging completions: {average_total_cars_finished_charging} ({average_charging_completions_per_car} per car).")
print(f"\n {all_simulations_negative_battery} cars did not have enough battery capacity across all simulations (average of: {avergae_total_negative_battery} per simulation.)")



Simulation 1 results:
Number of Finished Deliveries: 73
Total Orders: 100
Total waiting time of all cars until recharge: 14062.92006501396
Total recharged ammount of all cars in Wh: 629.0465578407411
Total charging requests: 7
Total charging completions: 1
0 cars had not enough battery capcity
27 unfinished orders and untouched orders: Order 74, Order 75, Order 76, Order 77, Order 78, Order 79, Order 80, Order 81, Order 82, Order 83, Order 84, Order 85, Order 86, Order 87, Order 88, Order 89, Order 90, Order 91, Order 92, Order 93, Order 94, Order 95, Order 96, Order 97, Order 98, Order 99, Order 100




Simulation 2 results:
Number of Finished Deliveries: 81
Total Orders: 100
Total waiting time of all cars until recharge: 12537.881376516942
Total recharged ammount of all cars in Wh: 665.4904284480028
Total charging requests: 7
Total charging completions: 1
0 cars had not enough battery capcity
19 unfinished orders and untouched orders: Order 79, Order 81, Order 82, Order 83, Order 84